ChemBERTa

In [5]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from transformers import AutoModel

# Step 1: 读取CSV文件并筛选不重复的药物SMILES
df = pd.read_csv('DrugcombDB.csv')  # 请根据实际文件路径修改
unique_smiles = set(df['DrugA']).union(set(df['DrugB']))  # 获取所有不重复的SMILES编码
print(len(unique_smiles))

# Step 2: 使用BERT对SMILES编码
tokenizer = AutoTokenizer.from_pretrained("ChemBERTa-zinc-base-v1")
model = AutoModel.from_pretrained("ChemBERTa-zinc-base-v1")

# 定义编码函数
def encode_smiles(smiles):
    tokens = tokenizer(smiles, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    # 使用[CLS]位置的embedding作为SMILES的表示
    return outputs.last_hidden_state[:, 0, :].numpy()

# Step 3: 编码所有SMILES并保存为字典
smiles_dict = {}
cnt = 0
for smile in unique_smiles:
    encoding = encode_smiles(smile)
    smiles_dict[smile] = encoding

    cnt += 1
    print(cnt)

# Step 4: 保存字典为npy文件
np.save('ChemBERTa_smiles_embeddings.npy', smiles_dict)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


210
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210


In [6]:
import pandas as pd

# 读取CSV文件
data = pd.read_csv('DrugcombDB.csv')

# 提取药物A和药物B的SMILES序列
drug_A_smiles = data['DrugA'].unique()  # 找到不重复的药物A序列
drug_B_smiles = data['DrugB'].unique()  # 找到不重复的药物B序列

# 合并药物A和药物B的SMILES序列，去重
unique_smiles = pd.Series(pd.concat([pd.Series(drug_A_smiles), pd.Series(drug_B_smiles)]).unique())

# 保存为单列CSV文件
unique_smiles.to_csv('unique_smiles.csv', index=False, header=['smiles'])

print("不重复的药物SMILES序列已保存到 'unique_smiles.csv'")

不重复的药物SMILES序列已保存到 'unique_smiles.csv'


In [7]:
import pandas as pd
import numpy as np

# 读取CSV文件
data = pd.read_csv('output_fn_210_encoding.csv')

# 提取 SMILES 列和 latent 向量
smiles = data['smiles'].values  # 获取SMILES序列
latent_vectors = data.drop(columns=['smiles']).values  # 去除smiles列和Unnamed列，获取latent向量

# 创建字典，smiles作为key，latent向量作为value
latent_dict = {sm: latent_vectors[i] for i, sm in enumerate(smiles)}

# 保存为npy文件
np.save('geminimol_latent_dict.npy', latent_dict)

print("SMILES和latent向量的字典已保存为 'geminimol_latent_dict.npy'", latent_dict)

SMILES和latent向量的字典已保存为 'geminimol_latent_dict.npy' {'C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1': array([-0.01528311,  0.00781033, -0.00911927, ..., -0.01021314,
       -0.01675927, -0.32612738]), 'C#Cc1cccc(Nc2ncnc3cc(OC)c(OCCCCCCC(=O)NO)cc23)c1': array([ 0.02334271,  0.03866177, -0.01690754, ...,  0.00187912,
       -0.00352233, -0.2994473 ]), 'C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1.Cl': array([ 0.03495072,  0.00323207,  0.00932615, ...,  0.03814946,
        0.0213738 , -0.23130028]), 'C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1': array([ 0.03387668,  0.00236321,  0.00978314, ...,  0.04166253,
        0.02244852, -0.23869732]), 'C1CN1c1nc(N2CC2)nc(N2CC2)n1': array([-0.06822942, -0.12278755,  0.08850537, ..., -0.08611048,
        0.04449665, -0.47344488]), 'C=C1C(=O)C23C(O)C1CCC2C12COC3(O)C(O)C1C(C)(C)CCC2O': array([ 0.06431489,  0.06555984,  0.00649654, ...,  0.01140895,
        0.00164388, -0.13146813]), 'C=C1CC2C(CCC3(C)C(=O)CCC23)C2(C)C=CC(=O)C=C12': array([ 